## Day 12: Hill Climbing Algorithm

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day12/puzzle.ipynb)

In [1]:
#!value --name sampleRaw 

Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi

In [2]:
#!value --name actualRaw --from-file ./data_actual.txt

In [3]:
type Node = 
    | Start
    | Finish
    | Default


In [4]:
#!share sampleRaw --from value
#!share actualRaw --from value
#load "../common/common.fsx"
let parse raw = 
    Pattern1.read (Seq.toArray) raw
    |> array2D
    |> Array2D.map (function | 'S' -> Start, 0 | 'E' -> Finish, 'z'-'a'|>int | c -> Default, c-'a'|>int)

let sampleMap = parse sampleRaw
let actualMap = parse actualRaw

In [5]:
#load "../common/matrixFormatting.fsx"
open System.Drawing

let toHeightColor (node, height) = 
    match node with 
    | Start -> Color.Blue
    | Finish -> Color.Red
    | Default -> 
        match height with
        | 0 -> Color.FromArgb(46, 97, 51)
        | 1 -> Color.FromArgb(56, 118, 64)
        | 2 -> Color.FromArgb(64, 134, 72)
        | 3 -> Color.FromArgb(69, 129, 70)
        | 4 -> Color.FromArgb(77, 121, 78)
        | 5 -> Color.FromArgb(89, 121, 91)
        | 6 -> Color.FromArgb(94, 118, 92)
        | 7 -> Color.FromArgb(97, 115, 96)
        | 8 -> Color.FromArgb(83, 83, 63)
        | 9 -> Color.FromArgb(99, 99, 75)
        | 10 -> Color.FromArgb(115, 115, 87)
        | 11 -> Color.FromArgb(123, 123, 94)
        | 12 -> Color.FromArgb(131, 131, 99)
        | 13 -> Color.FromArgb(142, 142, 108)
        | 14 -> Color.FromArgb(147, 147, 113)
        | 15 -> Color.FromArgb(151, 151, 117)
        | 16 -> Color.FromArgb(164, 164, 134)
        | 17 -> Color.FromArgb(171, 171, 143)
        | 18 -> Color.FromArgb(181, 181, 157)
        | 19 -> Color.FromArgb(199, 203, 204)
        | 20 -> Color.FromArgb(189, 203, 204)
        | 21 -> Color.FromArgb(179, 203, 204)
        | 22 -> Color.FromArgb(169, 203, 204)
        | 23 -> Color.FromArgb(159, 203, 204)
        | 24 -> Color.FromArgb(139, 203, 204)
        | 25 -> Color.FromArgb(129, 203, 204)
        | _ -> Color.Transparent
let heightMap heightMatrix = 
    heightMatrix |> toDisplayable toHeightColor

In [6]:
sampleMap |> heightMap

In [7]:
let actualMap = parse actualRaw

actualMap |> heightMap |> withSettings { Width = 600 }


### Part 1

This notebook relies on a breadth-first search algorithm implementation from the `bfs.fsx` script file. By default it treats adjacent elements in a matrix as adjacent in the graph, but this can be changed. The target can also be set up with an arbitrary function. The algotithm returns the whole path to the target, including the start and the target, and thus to calculate the step required we will need to decrement its length.

In [25]:
#load "../common/bfs.fsx"
open Bfs
open Bfs.Matrix

let adj : Adjacency<Node*int> = 
    Adjacencies.A4 
    |> Adjacencies.where (fun (_, height) (_, candidate) -> candidate - height <= 1)
let target1 : Target<_> = 
    fun _ (node, _) -> node = Finish

let findStart map = 
    map
    |> Array2D.indexed 
    |> Array2D.toSeq 
    |> Seq.find (fun ((i,j), (node, _)) -> node = Start)
    |> fst

let extractPath (Found(states)) = 
    states |> List.map(fun s -> s.Coordinates, s.Value)
    
let samplePath1 = 
    findPath { Adjacency = adj } sampleMap (findStart sampleMap) target1 |> extractPath

In [26]:
let actualPath1 = 
    findPath { Adjacency = adj } actualMap (findStart actualMap) target1 |> extractPath

To visualize the maps with the paths we might want to draw smaller "pixels" for the steps to achieve some dotted line feeling. Without changing our helpers, this can be done by replicating the map matrices, thus increasing the "pixel" density.

In [27]:
let withPathDisplayable map path =
    let newMap = Array2D.create (3 * Array2D.length1 map) (3 * Array2D.length2 map) System.Drawing.Color.Empty
    
    let pathMap = Array2D.create (Array2D.length1 newMap) (Array2D.length2 newMap) System.Drawing.Color.Empty
    path
    |> List.iter (fun ((i,j), _) -> 
        pathMap[i*3+1,j*3+1] <- System.Drawing.Color.FromArgb(200, 0, 0, 0)
    )
    
    newMap 
    |> Array2D.mapi (fun i j value -> toHeightColor map[(i/3), (j/3)])
    |> toDisplayable id
    |> withMatrix pathMap id


withPathDisplayable sampleMap samplePath1 |> display
List.length samplePath1 - 1

31

In [28]:
withPathDisplayable actualMap actualPath1
|> withSettings { Width = 600 }
|> display
List.length actualPath1 - 1

339

To achieve a more natural path to south-west of the mountain, we can even randomize the adjacencies!

In [30]:
let rnd = Random()
let adjRandom : Adjacency<Node*int> = 
    fun (i,j) value m ->
        adj (i,j) value m
        |> List.sortBy (fun _ -> rnd.Next())

let actualPath1' = findPath { Adjacency = adjRandom } actualMap (findStart actualMap) target1 |> extractPath

withPathDisplayable actualMap actualPath1'
|> withSettings { Width = 600 }
|> display

List.length actualPath1' - 1

339

### Part 2

For Part 2, we can start from the top in BF-search for any low-height node. At the same time, the adjacency function must be re-evaluated since we are about to change the direction.

In [32]:
let findFinish map = 
    map
    |> Array2D.indexed 
    |> Array2D.toSeq 
    |> Seq.find (fun ((i,j), (node, _)) -> node = Finish)
    |> fst

let adjBackward : Adjacency<Node*int> = 
    Adjacencies.A4 
    |> Adjacencies.where (fun (_, height) (_, candidate) -> height - candidate <= 1)
    
let target2 : Target<_> = 
    fun _ (_, height) -> height = 0

let samplePath2 = findPath { Adjacency = adjBackward } sampleMap (findFinish sampleMap) target2 |> extractPath

In [33]:
withPathDisplayable sampleMap samplePath2 |> display
List.length samplePath2 - 1

29

In [35]:
let actualPath2 = findPath { Adjacency = adjBackward } actualMap (findFinish actualMap) target2 |> extractPath


In [36]:
withPathDisplayable actualMap actualPath2
|> withSettings { Width = 600 }
|> display

List.length actualPath2 - 1

332